In [1]:
import pandas as pd
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
import matplotlib.pyplot as plt
import string
import re

In [2]:
df = pd.read_csv('data/judge-1377884607_tweet_product_company.csv', encoding='latin-1')
df.drop(index=6, inplace=True)
df = df.reset_index(drop=True)
df.head(10)

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
5,@teachntech00 New iPad Apps For #SpeechTherapy...,NaN,No emotion toward brand or product
6,"#SXSW is just starting, #CTIA is around the co...",Android,Positive emotion
7,Beautifully smart and simple idea RT @madebyma...,iPad or iPhone App,Positive emotion
8,Counting down the days to #sxsw plus strong Ca...,Apple,Positive emotion
9,Excited to meet the @samsungmobileus at #sxsw ...,Android,Positive emotion


In [3]:
df['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

No emotion toward brand or product    5388
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [4]:
df['emotion_in_tweet_is_directed_at'].value_counts()

iPad                               946
Apple                              661
iPad or iPhone App                 470
Google                             430
iPhone                             297
Other Google product or service    293
Android App                         81
Android                             78
Other Apple product or service      35
Name: emotion_in_tweet_is_directed_at, dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9092 entries, 0 to 9091
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9092 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


In [6]:
apple_list = ['iPad', 'Apple', 'iPad or iPhone App', 'Other Apple product or service', 'iPhone']
google_list = ['Google', 'Other Google product or service', 'Android App', 'Android']

df['emotion_in_tweet_is_directed_at'] = ['Apple' if val in apple_list
                                          else val for val in df['emotion_in_tweet_is_directed_at']]

df['emotion_in_tweet_is_directed_at'] = ['Google' if val in google_list
                                          else val for val in df['emotion_in_tweet_is_directed_at']]

df['emotion_in_tweet_is_directed_at'].value_counts()

Apple     2409
Google     882
Name: emotion_in_tweet_is_directed_at, dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9092 entries, 0 to 9091
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9092 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


In [8]:
df['emotion_in_tweet_is_directed_at'] = [val if val == 'Apple' or val == 'Google'
                                          else 'Google' if any(ele.casefold() in df['tweet_text'][ind].casefold() for ele in google_list)
                                          else 'Apple' if any(ele.casefold() in df['tweet_text'][ind].casefold() for ele in apple_list)
                                          else val
                                          for ind, val in df['emotion_in_tweet_is_directed_at'].iteritems()]

df['emotion_in_tweet_is_directed_at'].value_counts()

Apple     5346
Google    2985
Name: emotion_in_tweet_is_directed_at, dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9092 entries, 0 to 9091
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     8331 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9092 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


In [10]:
df = df.dropna(axis = 0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8331 entries, 0 to 9091
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          8331 non-null   object
 1   emotion_in_tweet_is_directed_at                     8331 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  8331 non-null   object
dtypes: object(3)
memory usage: 260.3+ KB


In [11]:
df

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,Apple,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,Apple,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,Apple,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,Apple,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
...,...,...,...
9087,Ipad everywhere. #SXSW {link},Apple,Positive emotion
9088,"Wave, buzz... RT @mention We interrupt your re...",Google,No emotion toward brand or product
9089,"Google's Zeiger, a physician never reported po...",Google,No emotion toward brand or product
9090,Some Verizon iPhone customers complained their...,Apple,No emotion toward brand or product


In [12]:
from nltk.tokenize import RegexpTokenizer 
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

tokenizer = RegexpTokenizer(r"(?u)\w{3,}")

# Create a list of stopwords in English
stopwords_list = stopwords.words('english')
# Create an instance of nltk's PorterStemmer with the variable name `stemme
stemmer = SnowballStemmer(language="english")


In [13]:
def preprocess_text(text, tokenizer, stopwords_list, stemmer): # Standardize case (lowercase the text)
# your code here
    lower = text.lower()
# Tokenize text using `tokenizer`
# your code here
    token_list = tokenizer.tokenize(lower)
# Remove stopwords using `stopwords_list`
# your code here
    remove_sw = [token for token in token_list if token not in stopwords_list]
# Stem the tokenized text using `stemmer`
# your code here
    final = [stemmer.stem(token) for token in remove_sw]
# Return the preprocessed text # your code here
    return final

In [14]:
text_series = df["tweet_text"].map(lambda x : preprocess_text(x, tokenizer, stopwords_list, stemmer))

In [15]:
# df["tweet_text"][3]
text_series[3]

['sxsw', 'hope', 'year', 'festiv', 'crashi', 'year', 'iphon', 'app', 'sxsw']

In [16]:
df2 = df.copy()

df2["tokenized_text"] = text_series

In [17]:
df2 = df2.drop(columns = "tweet_text")

In [18]:
df2 = df2.rename(columns={"emotion_in_tweet_is_directed_at": "product",\
                   "is_there_an_emotion_directed_at_a_brand_or_product":"emotion"})


In [19]:
df2 = df2[["tokenized_text", "product", 'emotion']]


In [20]:
df2

,tokenized_text,product,emotion
0,"[wesley83, iphon, hrs, tweet, rise_austin, dea...",Apple,Negative emotion
1,"[jessede, know, fludapp, awesom, ipad, iphon, ...",Apple,Positive emotion
2,"[swonderlin, wait, ipad, also, sale, sxsw]",Apple,Positive emotion
3,"[sxsw, hope, year, festiv, crashi, year, iphon...",Apple,Negative emotion
4,"[sxtxstate, great, stuff, fri, sxsw, marissa, ...",Google,Positive emotion
...,...,...,...
9087,"[ipad, everywher, sxsw, link]",Apple,Positive emotion
9088,"[wave, buzz, mention, interrupt, regular, sche...",Google,No emotion toward brand or product
9089,"[googl, zeiger, physician, never, report, pote...",Google,No emotion toward brand or product
9090,"[verizon, iphon, custom, complain, time, fell,...",Apple,No emotion toward brand or product


In [21]:
df2["product"].value_counts(normalize = True)

Apple     0.6417
Google    0.3583
Name: product, dtype: float64

Model to find what emotion these tokenized texts refer to. Do we need to undersample to fix class imbalance? 
Or does that not matter since we are only caring about Apple rather than Apple + Google ?
